In [1]:
import pandas as pd
import psutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
dataset = pd.read_csv("alternative_dataset_for_training.csv")
dataset = dataset.drop('Unnamed: 0', axis = 1)
dataset = dataset.drop(['adviceUrgence.1','triage_urgentie_code.1'],axis = 1)


In [2]:
# Define the bin edges for the 5-year intervals (e.g., 0-5, 5-10, ..., 70-75, etc.)
bins = list(range(0, 105, 5))  # Change 80 to a higher value if needed

# Create labels for each bin (e.g., '0-5', '5-10', etc.)
bin_labels = [f'{i}-{i+5}' for i in range(0, 100, 5)]

# Use pd.cut to create a new column 'age_category' based on the bins
dataset['age_category'] = pd.cut(dataset['age'], bins=bins, labels=bin_labels, right=False)

# One-hot encode the 'age_category' column
df_one_hot = pd.get_dummies(dataset['age_category'], prefix='age')

# Join the one-hot encoded columns back to the original dataframe
dataset = pd.concat([dataset, df_one_hot], axis=1)

# Drop the original 'age_category' column if not needed
dataset.drop('age_category', axis=1, inplace=True)
dataset = dataset.drop('age', axis =1)
#dataset = dataset.loc[:,~dataset.columns.str.endswith('.1')].columns
dataset = dataset[dataset.loc[:,~dataset.columns.str.endswith('.1')].columns]

In [3]:
vertaal_df = pd.read_csv("UTF-8Vertaling_Ids_Vragen").drop("Unnamed: 0", axis = 1)
vertaal_df['Id'] = "Q_"+vertaal_df['Id'].astype(str)
vertaal_df
mapping = {row["Id"]: f'{row["Title"]} ({row["Id"]})' for _, row in vertaal_df.iterrows()}

# Rename the columns in dataset
dataset = dataset.rename(columns=mapping)

In [4]:
def Maak_Lokale_Dataset(Triage_MINDD, Escalation, Dataset, Lichaamsdeel= None):
    Local_subset = Dataset.copy()
    Local_subset['adviceUrgence'] = Local_subset['adviceUrgence'].str.replace('U', '').astype(int)
    Local_subset['triage_urgentie_code'] = Local_subset['triage_urgentie_code'].str.replace('U', '').astype(int)
    if Escalation == "DeEscalation":
        Local_subset =  Local_subset[((Local_subset['adviceUrgence'].astype(int) == Triage_MINDD) & (Local_subset['triage_urgentie_code'].astype(int) >= Triage_MINDD))]
        Local_subset['Escalatie'] = Local_subset['triage_urgentie_code'].apply(lambda x:-1 if x<Triage_MINDD else (0 if x==Triage_MINDD else 1))

    if Escalation == "Escalation":
        Local_subset =  Local_subset[((Local_subset['adviceUrgence'].astype(int) == Triage_MINDD) & (Local_subset['triage_urgentie_code'].astype(int) <= Triage_MINDD))]
        Local_subset['Escalatie'] = Local_subset['triage_urgentie_code'].apply(lambda x:1 if x<Triage_MINDD else (0 if x==Triage_MINDD else -1))

    Local_subset = Local_subset.loc[:, (Local_subset != 0).any(axis=0)]
    if Lichaamsdeel != None:
        lichaamsdeelnaam = "bodyAreaTitle_"+Lichaamsdeel
        Local_subset = Local_subset[Local_subset[lichaamsdeelnaam]==1]
    Local_subset = Local_subset.replace({"no":-1,"not asked":0,"yes":1})
    Local_subset = Local_subset.loc[:,(Local_subset != 0).any(axis =0)]
    Local_subset = Local_subset.replace({False:0,True:1})
    # Substring to search for in column names
    substring = 'Q_'  # Change this to your desired substring
    
    # Filter columns whose names contain the substring
    columns_to_filter = [col for col in Local_subset.columns if substring in col]

    # Count occurrences of 1 and -1 in the selected columns
    count_ones_and_negatives = Local_subset[columns_to_filter].apply(lambda col: (col == 1).sum() + (col == -1).sum())

    # Keep only columns where the count of 1s and -1s is at least 2
    columns_to_drop = count_ones_and_negatives[count_ones_and_negatives < 5].index

    # Drop the filtered columns
    df_filtered = Local_subset.drop(columns=columns_to_drop)

    return df_filtered
    return Local_subset

In [5]:
def Create_model(Triage_MINDD, Escalation, X_train, y_train):
    from sklearn.linear_model import LogisticRegression    
    logreg = LogisticRegression(max_iter = 100000, class_weight = "balanced", solver = "liblinear")
    logreg.fit(X_train, y_train)
    return logreg

In [6]:
def Create_model_SVM(Triage_MINDD, Escalation, X_train, y_train):
    from sklearn import svm
    svm = svm.SVC(kernel = 'linear', C=0.1, max_iter = -1, gamma = 10, class_weight = "balanced")
    svm.fit(X_train, y_train)
    return svm

In [7]:
def Create_model_EBM(Triage_MINDD, Escalation, X_train, y_train):
    from interpret.glassbox import ExplainableBoostingClassifier
    ebm = ExplainableBoostingClassifier(feature_types = ['nominal' for number in range(len(X_train.columns))],smoothing_rounds = 100,max_rounds = 1000000000, learning_rate = 0.02, interactions = 0.99, interaction_smoothing_rounds = 5, inner_bags = 20)
    from sklearn.utils.class_weight import compute_sample_weight
    weights = compute_sample_weight(class_weight='balanced', y=y_train)
    ebm.fit(X_train, y_train, sample_weight = weights)
    return ebm

In [8]:
def Create_model_DT(Triage_MINDD, Escalation, X_train, y_train):
    from sklearn import tree
    dt = tree.DecisionTreeClassifier(max_depth = 5, min_samples_leaf = 5, class_weight = "balanced")
    dt.fit(X_train, y_train)
    return dt

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, lasso_path
from sklearn.metrics import roc_auc_score

# Assume X_train, X_test, X_validation, y_train, y_test, y_validation are defined and share the same features
def logistic_regression_simplification(model, X_train, X_test, y_train, y_test): 
# Train an SVM with a linear kernel
    logreg = model
    #
    # Get feature weights from the trained linear SVM
    coef = logreg.coef_.copy()

    # Get raw term contributions
    X_trn_tc = X_train * coef  # Element-wise multiplication
    X_tst_tc = X_test * coef

    # LASSO path over logreg term contributions
    alphas, coefs, _ = lasso_path(X_trn_tc, y=y_train, positive=True)

    # Evaluate each alpha using AUC-ROC
    aucs = []
    for coef_lasso in coefs.T:
        lasso_pred = X_tst_tc @ coef_lasso
        auc = roc_auc_score(y_test, lasso_pred)
        aucs.append(auc)

    # Select best alpha based on max AUC
    best_idx = np.argmax(aucs)
    best_coef = coefs[:, best_idx]

    # Identify selected features (non-zero LASSO coefficients)
    non_zero_mask = best_coef != 0
    selected_features = X_train.columns[non_zero_mask]

    # Filter datasets to keep only selected features
    X_train_filtered = X_train[selected_features]
    X_test_filtered = X_test[selected_features]

    # Retrain SVM on filtered dataset
    logreg_filtered = LogisticRegression(max_iter = 100000, solver = "liblinear",class_weight = "balanced")
    logreg_filtered.fit(X_train_filtered, y_train)
    
    return logreg_filtered, X_test_filtered

In [10]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import Lasso, lasso_path
from sklearn.metrics import roc_auc_score

# Assume X_train, X_test, X_validation, y_train, y_test, y_validation are defined and share the same features
def SVM_simplification(model, X_train, X_test, y_train, y_test): 

    svm = model
    # Get feature weights from the trained linear SVM
    coef = svm.coef_.copy()

    # Get raw term contributions
    X_trn_tc = X_train * coef  # Element-wise multiplication
    X_tst_tc = X_test * coef

    # LASSO path over SVM term contributions
    alphas, coefs, _ = lasso_path(X_trn_tc, y=y_train, positive=True)

    # Evaluate each alpha using AUC-ROC
    aucs = []
    for coef_lasso in coefs.T:
        lasso_pred = X_tst_tc @ coef_lasso
        auc = roc_auc_score(y_test, lasso_pred)
        aucs.append(auc)

    # Select best alpha based on max AUC
    best_idx = np.argmax(aucs)
    best_coef = coefs[:, best_idx]

    # Identify selected features (non-zero LASSO coefficients)
    non_zero_mask = best_coef != 0
    selected_features = X_train.columns[non_zero_mask]

    # Filter datasets to keep only selected features
    X_train_filtered = X_train[selected_features]
    X_test_filtered = X_test[selected_features]

    # Retrain SVM on filtered dataset
    svm_filtered = SVC(kernel = 'linear', C=0.1, max_iter = -1, gamma = 10, class_weight = "balanced")
    svm_filtered.fit(X_train_filtered, y_train)

    return svm_filtered, X_test_filtered

In [11]:
import numpy as np
import pandas as pd
import copy
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold
def simplify_ebm_with_logistic_reweighting(ebm_model,X_train,X_test,y_train,y_test,C_grid=None,keep_intercept=True,verbose=True,cv_folds=5):
    C_grid = np.logspace(-3, 2, 10)  # From 0.001 to 100

    # Get EBM term contributions
    X_train_terms = ebm_model.eval_terms(X_train)
    X_test_terms = ebm_model.eval_terms(X_test)

    # === Cross-validation to find best C ===
    best_C = None
    best_score = -np.inf
    auc_scores = []

    for C in C_grid:
        clf = LogisticRegression(
        penalty='l1',
        solver='liblinear',
        class_weight='balanced',
        C=C,
        max_iter=1000
    )
    clf.fit(X_fold_train, y_fold_train)
    y_pred = clf.predict_proba(X_fold_val)[:, 1]

    if len(np.unique(y_fold_val)) < 2:
        continue  # Skip invalid fold

    auc = roc_auc_score(y_fold_val, y_pred)
    auc_scores.append(auc)

    # === Refit with best C on full training set ===
    clf = LogisticRegression(
        penalty='l1',
        solver='liblinear',
        class_weight='balanced',
        C=best_C,
        max_iter=1000
    )
    clf.fit(X_train_terms, y_train)
    coef = clf.coef_[0]
    # Clone the model
    simplified_ebm = copy.deepcopy(ebm_model)

    # Apply weights to EBM term functions
    for idx, weight in enumerate(coef):
        simplified_ebm.scale(idx, factor=weight)

    simplified_ebm.intercept_ = float(clf.intercept_)

    simplified_ebm.sweep()
    return simplified_ebm


In [12]:
from sklearn import tree
def simplifiy_decision_trees(model, X_train, X_test, y_train, y_test):
    DT = model
    models = []
    path = DT.cost_complexity_pruning_path(X_train,y_train)
    alphas, impurities = abs(path.ccp_alphas), path.impurities
    for alpha in alphas:
        clf = tree.DecisionTreeClassifier(ccp_alpha=alpha, max_depth = 5,min_samples_leaf = 5, class_weight = "balanced")
        clf.fit(X_train, y_train)
        models.append(clf)
    test_scores = [roc_auc_score(y_test,clf.predict(X_test)) for clf in models]
    max_score = np.max(test_scores)
    candidate_alphas = alphas[test_scores == max_score]
    best_alpha = np.max(candidate_alphas)
    simplified_DT = tree.DecisionTreeClassifier(ccp_alpha = best_alpha, max_depth = 5, min_samples_leaf = 5, class_weight = "balanced")
    simplified_DT.fit(X_train, y_train)
    return simplified_DT
    

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
def custom_10_fold(dataset, model_type, Triage_MINDD, Escalation):
    local_dataset = Maak_Lokale_Dataset(Triage_MINDD, Escalation, dataset)
    kf = KFold(n_splits = 10)
    X_data = local_dataset.drop(['triage_urgentie_code','adviceUrgence','Escalatie'],axis=1)
    y_data = local_dataset['Escalatie']
    roc_scores = []
    f1_scores = []
    recall_scores = []
    roc_simplified_scores = []
    f1_simplified_scores = []
    recall_simplified_scores = []
    n_params = []
    n_params_simplified = []
    for train_index, test_index in kf.split(local_dataset):
        X_train, X_test = X_data.iloc[train_index], X_data.iloc[test_index]
        y_train, y_test = y_data.iloc[train_index], y_data.iloc[test_index]
        if model_type == "logreg":
            model = Create_model(Triage_MINDD, Escalation, X_train, y_train)
            predictions = model.predict(X_test)
            f1 = f1_score(y_test, predictions)
            recall = recall_score(y_test, predictions)
            roc_auc = roc_auc_score(y_test, predictions)
            f1_scores.append(f1)
            recall_scores.append(recall )
            roc_scores.append(roc_auc)
            n_param = len(model.coef_[0])
            n_params.append(n_param) 
            simplified_model, X_test_filtered = logistic_regression_simplification(model,X_train, X_test, y_train, y_test)
            simplified_predictions = simplified_model.predict(X_test_filtered)
            f1_simplified = f1_score(y_test, simplified_predictions)
            recall_simplified = recall_score(y_test, simplified_predictions)
            roc_auc_simplified = roc_auc_score(y_test, simplified_predictions)
            f1_simplified_scores.append(f1_simplified)
            recall_simplified_scores.append(recall_simplified)
            roc_simplified_scores.append(roc_auc_simplified)
            n_param_simplified = len(simplified_model.coef_[0])
            n_params_simplified.append(n_param_simplified)
        if model_type == "SVM":
            model = Create_model_SVM(Triage_MINDD, Escalation, X_train, y_train)
            predictions = model.predict(X_test)
            f1 = f1_score(y_test, predictions)
            recall = recall_score(y_test, predictions)
            roc_auc = roc_auc_score(y_test, predictions)
            f1_scores.append(f1)
            recall_scores.append(recall)
            roc_scores.append(roc_auc)
            n_param = len(model.coef_[0])
            n_params.append(n_param) 
            simplified_model, X_test_filtered = SVM_simplification(model, X_train,X_test, y_train, y_test)
            simplified_predictions = simplified_model.predict(X_test_filtered)
            f1_simplified = f1_score(y_test, simplified_predictions)
            recall_simplified = recall_score(y_test, simplified_predictions)
            roc_auc_simplified = roc_auc_score(y_test, simplified_predictions)
            f1_simplified_scores.append(f1_simplified)
            recall_simplified_scores.append(recall_simplified)
            roc_simplified_scores.append(roc_auc_simplified)
            n_param_simplified = len(simplified_model.coef_[0])
            n_params_simplified.append(n_param_simplified)
        if model_type == "DT":
            model = Create_model_DT(Triage_MINDD, Escalation, X_train, y_train)
            predictions = model.predict(X_test)
            f1 = f1_score(y_test, predictions)
            recall = recall_score(y_test, predictions)
            roc_auc = roc_auc_score(y_test, predictions)
            f1_scores.append(f1)
            recall_scores.append(recall)
            roc_scores.append(roc_auc)
            n_param = sum(model.tree_.children_left != -1)
            n_params.append(n_param)
            simplified_model = simplifiy_decision_trees(model, X_train,X_test, y_train, y_test)
            simplified_predictions = simplified_model.predict(X_test)
            f1_simplified = f1_score(y_test, simplified_predictions)
            recall_simplified = recall_score(y_test, simplified_predictions)
            roc_auc_simplified = roc_auc_score(y_test, simplified_predictions)
            f1_simplified_scores.append(f1_simplified)
            recall_simplified_scores.append(recall_simplified)
            roc_simplified_scores.append(roc_auc_simplified)
            n_param_simplified = sum(simplified_model.tree_.children_left != -1)
            n_params_simplified.append(n_param_simplified)
        if model_type == "EBM":
            model = Create_model_EBM(Triage_MINDD, Escalation, X_train, y_train)
            predictions = model.predict(X_test)
            f1 = f1_score(y_test, predictions)
            recall = recall_score(y_test, predictions)
            roc_auc = roc_auc_score(y_test, predictions)
            f1_scores.append(f1)
            recall_scores.append(recall)
            roc_scores.append(roc_auc)
            n_param = len(model.term_names_)
            n_params.append(n_param)
            simplified_model = simplify_ebm_with_logistic_reweighting(model, X_train,X_test, y_train, y_test)
            simplified_predictions = simplified_model.predict(X_test)
            f1_simplified = f1_score(y_test, simplified_predictions)
            recall_simplified = recall_score(y_test, simplified_predictions)
            roc_auc_simplified = roc_auc_score(y_test, simplified_predictions)
            f1_simplified_scores.append(f1_simplified)
            recall_simplified_scores.append(recall_simplified)
            roc_simplified_scores.append(roc_auc_simplified)
            n_param_simplified = len(simplified_model.term_names_)
            n_params_simplified.append(n_param_simplified)
            
    return roc_scores, f1_scores, recall_scores, n_params, roc_simplified_scores, f1_simplified_scores, recall_simplified_scores, n_params_simplified

In [ ]:
import itertools
import csv
all_possible_combinations = [(1, "DeEscalation"),(2,"DeEscalation"),(2,"Escalation"),(3,"DeEscalation"),(3,"Escalation"),(4, "DeEscalation"),(4, "Escalation"),(5,"Escalation")]
all_model_types = ["logreg"]
for type in all_model_types:
    all_roc_scores = []
    all_f1_scores = []
    all_recall_scores = []
    all_n_params = []
    all_roc_simplified_scores = []
    all_f1_simplified_scores = []
    all_recall_simplified_scores = []
    all_n_params_simplified = []
    
    for combo in all_possible_combinations:
        print(combo)
        roc_scores, f1_scores, recall_scores, n_params, roc_simplified_scores, f1_simplified_scores, recall_simplified_scores, n_params_simplified = custom_10_fold(dataset, type, combo[0], combo[1])
        all_roc_scores.append(roc_scores)
        all_f1_scores.append(f1_scores)
        all_recall_scores.append(recall_scores)
        all_n_params.append(n_params)
        all_roc_simplified_scores.append(roc_simplified_scores)
        all_f1_simplified_scores.append(f1_simplified_scores)
        all_recall_simplified_scores.append(recall_simplified_scores)
        all_n_params_simplified.append(n_params_simplified)
    file_name = str(type) +"_fold_results.csv"
    rows = zip(all_possible_combinations,all_roc_scores,all_f1_scores,all_recall_scores,all_n_params, all_roc_simplified_scores, all_f1_simplified_scores, all_recall_simplified_scores,all_n_params_simplified)
    with open(file_name,'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['model','all_roc_scores','all_f1_scores','all_recall_scores','all_n_params','all_roc_simplified_scores', 'all_f1_simplified_scores','all_recall_simplified_scores','all_n_params_simplified'])
        csv_out.writerows(rows)